# Libraries

In [85]:
import os
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from ipywidgets import interact, interact_manual
from bokeh.io import output_notebook
from bokeh.models import Legend, LegendItem
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
output_notebook()
from numpy import pi
from bokeh.transform import cumsum
from bokeh.models.widgets import Panel, Tabs
from bokeh.palettes import Spectral
from bokeh.palettes import RdBu3
from bokeh.palettes import Category20c

Loading BokehJS ...

# Functions

In [86]:
def load_df(dict):
    df_dict = {}
    for key in dict.keys():
        df_dict['df_{}'.format(key)]=pd.read_csv(os.path.join(path_dict[key]))
    return df_dict

In [87]:
def merge_codes(df1,df_cod, left_column_key):
    merged_df = df1.merge(df_cod.drop(columns = ['NOTES']),
                          how = 'left',
                          left_on = left_column_key,
                          right_on = 'CODE',
                          indicator = True).rename(
                                                    columns={'ACTION_TAKEN': '{}_action'.format(left_column_key)}).drop(
                                                                                                                    columns = 'CODE')
    merge_result = merged_df['_merge'].value_counts()
    print('{} observations were matched, representing {}% of total observations'.format(merge_result[2],
                                                                                        round((merge_result[2]/len(merged_df)*100),2)))
    return merged_df.drop(columns = '_merge')

In [88]:
def merge_accused_to_codes(df1, df_cod):
    df_first_step = merge_codes(df1, df_cod, 'recommended_discipline')
    df_second_step = merge_codes(df_first_step, df_cod, 'final_discipline')
    return df_second_step

In [89]:
def merge_df(df1,df2):
    new_df = df1.merge(df2, how = 'outer', on = 'cr_id', indicator = True, suffixes = ('_accused', '_investigator'))
    merge_result = new_df['_merge'].value_counts()
    print('{} observations were matched, representing {}% of the new dataframe'.format(merge_result[2],
                                                                                        round((merge_result[2]/len(new_df)*100),2)))
    return new_df.drop(columns = '_merge')

In [90]:
def filter_unique_cr_id(df):
    duplicates_identifier = df.duplicated(subset=['cr_id','UID_accused'], keep= 'first')
    #include a counter of the number of duplicate cr_id that are removed
    df_unique_complaints = df[duplicates_identifier == False]
    return df_unique_complaints

In [91]:
def get_relative_freq(df):
    total_count=df['Count'].sum()
    df['Relative_count']=df['Count'].map(lambda x: round((x/total_count)*100,2))
    return df

In [92]:
def filter_nan(df, columns):
    filtered_df = df
    for i in columns:
        filtered_df[i] = df[df[i].notna()]
    return filtered_df

In [93]:
def histogram_df(df, complaint_cat='total'):
    df = filter_unique_cr_id(df)
    if complaint_cat != 'total':
        df = df[df['complaint_category'] == complaint_cat]
    else:
        pass
    final_df = df[df['age_victim'].notna()]
    arr_hist, edges = np.histogram(final_df['age_victim'], # Source: https://bit.ly/2D7lu87
                               bins = 106,
                               range = (0,106))
    hist_df = pd.DataFrame({'Count':   arr_hist, 
                            'left':  edges[:-1], 
                            'right': edges[1:]})
    hist_df = get_relative_freq(hist_df)
    return hist_df

In [94]:
def plot_histogram(df, complaint_cat):
    
    p = figure(plot_height = 600, plot_width = 600, 
           title = "Distribution of total complaints by victim's age",
           x_axis_label = "Victim's age", 
           y_axis_label = 'Percentage of observations over total')
    
    overall_hist_df = histogram_df(df)
    second_hist_df = histogram_df(df, complaint_cat)
    src_second_hist = ColumnDataSource(second_hist_df)
    
    selected_complaint = p.quad(source=src_second_hist,
                                bottom=0, top='Relative_count', 
                                left='left', right='right', 
                                fill_color='navy', line_color='white', alpha=0.7,
                                hover_fill_alpha = 1.0, hover_fill_color = 'red')
    
    all_complaints = p.line(x=overall_hist_df['right'] ,y=overall_hist_df['Relative_count'], 
           line_color="#ff8888", line_width=3, alpha=0.85) #Source: https://bit.ly/337c1bu and https://bit.ly/2XHmtFw
    
    legend = Legend(items=[("All complaints",[all_complaints]), ("{}".format(complaint_cat),[selected_complaint])],
                    location = 'top_right',
                    border_line_color="white") #Source: https://bit.ly/2KRNnFo and https://bit.ly/2DdMvqt    
    
    hover = HoverTool(tooltips=[('Age', '@right'), #Source: https://bit.ly/2Ok2ABk
                            ('Number of complaints', '@Count'),
                            ('Percentage of complaints', '@Relative_count'+'%')],
                     renderers=[selected_complaint])
    

    p.add_tools(hover)
    p.add_layout(legend)
    p.legend.label_text_font_size = '6pt'#Source: https://bit.ly/2QNRBla
    return p

In [95]:
def retrieve_top_complaint_types(df, n=20):
    df = filter_unique_cr_id(df)
    top_complaints_df = df['complaint_category'].value_counts()[0:n].reset_index()
    top_complaints = top_complaints_df['index']
    return top_complaints

In [96]:
def calculate_racial_proportion(race_of_accused):
    chart_colours = ["red", "green", "blue", "black", "orange", "blue"]
    df_race = merged_by_race_of_accused_and_victim_complaints.loc[merged_by_race_of_accused_and_victim_complaints['race_acc'] == race_of_accused]
    df_race['angle'] = df_race['share_in_total_complaints_by_race']*2*pi
    #df_race['color'] = chart_colours[len(df_race)]
    df_race['color'] = Category20c[len(df_race)]    
    plot_race = figure(plot_height=350, title= "Race-wise proportion of victims when officer is {}".format(race_of_accused), toolbar_location=None, tools="hover", tooltips = "@share_in_total_complaints_by_race:@race_invst_vic")
    plot_race.legend.label_text_font_size = '10pt'
    plot_race.wedge(x=0, y=1, radius=0.3, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'), line_color="white", fill_color='color', legend='race_invst_vic', source=df_race)
    plot_race.legend.location = "top_left"
    plot_race.legend.click_policy="hide"
    plot_race.legend.label_text_font_size = "6pt"    
  
    return plot_race

In [97]:
def calculate_gender_proportion(gender_of_accused):
    chart_colours = ["red", "green", "blue", "black", "orange", "blue"]
    c1 = RdBu3[2] # red
    c2 = RdBu3[0] # blue
    
    
    df_gender = merged_by_gender_of_accused_and_victim_complaints.loc[merged_by_gender_of_accused_and_victim_complaints['gender_acc'] == gender_of_accused]
    df_gender['angle'] = df_gender['share_in_total_complaints_by_gender']*2*pi
    #df_gender['color'] = chart_colours[len(df_gender)]
    df_gender['color'] = [c1, c2]
    plot_gender = figure(plot_height=350, title= "Gender-wise proportion of victims when officer is {}".format(gender_of_accused), toolbar_location=None, tools="hover", tooltips = "@share_in_total_complaints_by_gender:@gender_invst_vic")
    plot_gender.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'), line_color="white", fill_color='color', legend='gender_invst_vic', source=df_gender)
    
    plot_gender.legend.location = "top_left"
    plot_gender.legend.click_policy="hide"
    plot_gender.legend.label_text_font_size = "6pt"    
    
    return plot_gender

In [98]:
def prepare_data_race_pie_chart_df(df):
    total_df = df
    complaints_by_race_of_victim = total_df.groupby(['race_victim']).size().reset_index(name = 'count').dropna()
    total_complaints_by_race_of_victim = complaints_by_race_of_victim['count'].sum()
    complaints_by_race_of_victim['share in total complaints'] = complaints_by_race_of_victim['count']/total_complaints_by_race_of_victim
    complaints_by_race_of_accused_and_victim = total_df.groupby(['race_accused', 'race_victim']).size().reset_index(name = 'count').dropna()
    complaints_by_race_of_accused = total_df.groupby(['race_accused', 'race_victim']).size().reset_index(name = 'count').dropna().groupby(by = ['race_accused'])['count'].sum().reset_index(name = 'total complaints')
    merged_by_race_of_accused_and_victim_complaints = complaints_by_race_of_accused_and_victim.merge(complaints_by_race_of_accused, on = 'race_accused')
    merged_by_race_of_accused_and_victim_complaints['share_in_total_complaints_by_race'] = merged_by_race_of_accused_and_victim_complaints['count']/merged_by_race_of_accused_and_victim_complaints['total complaints']
    return merged_by_race_of_accused_and_victim_complaints
    

In [99]:
def prepare_data_gender_pie_chart_df(df):
    total_df = df
    complaints_by_gender_of_victim = total_df.groupby(['gender_victim']).size().reset_index(name = 'count').dropna()
    total_complaints_by_gender_of_victim = complaints_by_gender_of_victim['count'].sum()
    complaints_by_gender_of_victim['share in total complaints by gender'] = complaints_by_gender_of_victim['count']/total_complaints_by_gender_of_victim
    complaints_by_gender_of_accused_and_victim = total_df.groupby(['gender_accused', 'gender_victim']).size().reset_index(name = 'count').dropna()
    complaints_by_gender_of_accused = total_df.groupby(['gender_accused', 'gender_victim']).size().reset_index(name = 'count').dropna().groupby(by = ['gender_accused'])['count'].sum().reset_index(name = 'total complaints')
    merged_by_gender_of_accused_and_victim_complaints = complaints_by_gender_of_accused_and_victim.merge(complaints_by_gender_of_accused, on = 'gender_accused')
    merged_by_gender_of_accused_and_victim_complaints['share_in_total_complaints_by_gender'] = merged_by_gender_of_accused_and_victim_complaints['count']/merged_by_gender_of_accused_and_victim_complaints['total complaints']
    return merged_by_gender_of_accused_and_victim_complaints

In [100]:
def race_pie_chart(race_of_accused, race_pie_chart_df):
    #YOU NEVER DEFINED race_pie_chart_df
    df_race = race_pie_chart_df.loc[race_pie_chart_df['race_accused'] == race_of_accused]
    df_race['angle'] = df_race['share_in_total_complaints_by_race']*2*pi
    df_race['color'] = Category20c[len(df_race)]    
    plot_race = figure(plot_height=350, 
                       title= "Race-wise proportion of victims when officer is {}".format(race_of_accused), 
                       toolbar_location=None, 
                       tools="hover", 
                       tooltips = "@share_in_total_complaints_by_race:@race_victim")
    plot_race.legend.label_text_font_size = '10pt'
    plot_race.wedge(x=0, y=1, radius=0.3, 
                    start_angle=cumsum('angle', include_zero=True), 
                    end_angle=cumsum('angle'), 
                    line_color="white", 
                    fill_color='color', 
                    legend='race_victim', 
                    source=df_race)
    plot_race.legend.location = "top_left"
    plot_race.legend.click_policy="hide"
    plot_race.legend.label_text_font_size = "6pt"    
  
    return plot_race

In [101]:
def gender_pie_chart(gender_of_accused, gender_pie_chart_df):
    c1 = RdBu3[2] # red
    c2 = RdBu3[0] # blue

    df_gender = gender_pie_chart_df.loc[gender_pie_chart_df['gender_accused'] == gender_of_accused]
    df_gender['angle'] = df_gender['share_in_total_complaints_by_gender']*2*pi
    df_gender['color'] = [c1, c2]
    plot_gender = figure(plot_height=350, 
                         title= "Gender-wise proportion of victims when officer is {}".format(gender_of_accused), 
                         toolbar_location=None, 
                         tools="hover", 
                         tooltips = "@share_in_total_complaints_by_gender:@gender_victim")
    plot_gender.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), 
                      end_angle=cumsum('angle'), 
                      line_color="white", 
                      fill_color='color', 
                      legend='gender_victim', 
                      source=df_gender)
    
    plot_gender.legend.location = "top_left"
    plot_gender.legend.click_policy="hide"
    plot_gender.legend.label_text_font_size = "6pt"    
    
    return plot_gender

In [102]:
def load_data(working_dir, path):
    os.chdir(path_working_directory) # Source: https://bit.ly/2Og4HF0
    df_dict = load_df(path_dict)
    return df_dict
    

In [103]:
def merge_all_df(dic):
    keys=list(dic.keys())
    df_accused_and_codes = merge_accused_to_codes(dic[keys[1]], dic[keys[3]])
    df_accused_codes_victims = merge_df(df_accused_and_codes,dic[keys[0]])
    final_df = merge_df(df_accused_codes_victims, dic[keys[2]])
    total_df = final_df.merge(dic[keys[4]], 
                       how='left', 
                       left_on = 'UID_accused', 
                       right_on = 'UID', 
                       indicator = True,
                       suffixes = ('_investigator','_accused'))
    total_df = total_df.rename(columns = {'age': 'age_victim',
                             'gender_investigator': 'gender_victim',
                             'race_investigator': 'race_victim',
                             'UID': 'UID_profile'})
    return total_df
    
    
    
    

In [111]:
def race_gender_tab_graph(df):
    
    race_pie_chart_df = prepare_data_race_pie_chart_df(df)
    gender_pie_chart_df = prepare_data_gender_pie_chart_df(df)
    
    genders = list(gender_pie_chart_df.reset_index()['gender_accused'].unique())
    races = list(race_pie_chart_df.reset_index()['race_accused'].unique())
    
    race_plots = [race_pie_chart(race, race_pie_chart_df) for race in races]
    race_gridplot = gridplot([[race_plots[0], race_plots[1]],
                              [race_plots[2], race_plots[3]],
                              [race_plots[4], None]])
    race_panel = Panel(child = race_gridplot, title = 'Race-wise breakup of complaints')
    
    gender_plots = [gender_pie_chart(gender,gender_pie_chart_df) for gender in genders]
    gender_gridplot = gridplot([[gender_plots[0], gender_plots[1], None]])
    gender_panel = Panel(child = gender_gridplot, title = 'Gender-wise breakup of complaints')
    
    tabs = Tabs(tabs=[race_panel, gender_panel])
    show(tabs)
    

In [112]:
def main():
    df_dict = load_data(path_working_directory,path_dict )
    total_df = merge_all_df(df_dict)
    # Graph 1
    complaint_categories = retrieve_top_complaint_types(total_df, n=10)
    @interact(Complaint=complaint_categories)
    def make_plot_for(Complaint=complaint_categories[0]):
        plot = plot_histogram(total_df, Complaint)
        show(plot)
    #Graph 2
    race_gender_tab_graph(total_df)     

# Execution

In [113]:
path_working_directory = r'C:\Users\adminuser\Documents\GitHub\assignment-5-javier_prabhat_assign5'

path_dict = {'victims':     'used_data/complaints-victims_2000-2016_2016-11.csv',
            'accused':      'used_data/complaints-accused_2000-2016_2016-11.csv',
            'investigators':'used_data/complaints-investigators_2000-2016_2016-11.csv',
            'codes':        'used_data/discipline_penalty_codes.csv',
            'profiles':     'used_data/final-profiles.csv'}

In [114]:
main()

C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


118549 observations were matched, representing 94.4% of total observations
117870 observations were matched, representing 93.86% of total observations
28831 observations were matched, representing 20.73% of the new dataframe
166829 observations were matched, representing 79.3% of the new dataframe


interactive(children=(Dropdown(description='Complaint', options=('10U-INADEQUATE/FAILURE TO PROVIDE SERVICE', …

C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

# Graph 1: histogram of most frequent complaint types by victim's age (also shown here for better display)

In [115]:
df_dict = load_data(path_working_directory,path_dict )
total_df = merge_all_df(df_dict)

C:\Users\adminuser\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


118549 observations were matched, representing 94.4% of total observations
117870 observations were matched, representing 93.86% of total observations
28831 observations were matched, representing 20.73% of the new dataframe
166829 observations were matched, representing 79.3% of the new dataframe


In [116]:
complaint_categories = retrieve_top_complaint_types(total_df, n=10)
#total_df WAS ONLY EVER DEFINED WITHIN FUNCTIONS SO IT DOES NOT EXIST TO PASS INTO AN ARGUMENT. 
# I HAD TO MANUALLY DETERMINE HOW TO GET YOUR total_df. ALWAYS MAKE SURE YOUR CODE RUNS BEFORE SUBMITTING.

In [117]:
@interact(Complaint=complaint_categories)
def make_plot_for(Complaint=complaint_categories[0]):
    plot = plot_histogram(total_df, Complaint)
    show(plot)

interactive(children=(Dropdown(description='Complaint', options=('10U-INADEQUATE/FAILURE TO PROVIDE SERVICE', …

# Graph 2: Racial distribution of complaints by race and gender of accused officer (also shown here for better display)

In [118]:
race_gender_tab_graph(total_df)

C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\adminuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa